In [1]:
import os
import numpy as np

token_len = 4
data_dir = './data/aa%i/' % token_len

data_filename = 'data.npy'
indices_filename = 'indices.npy'
indptr_filename = 'indptr.npy'
labels_filename = 'labels.npy'
shape_filename = 'shape.npy'
vocab_filename = 'vocab.npy'

feature_dtype = np.int8

data = np.load(os.path.join(data_dir, data_filename)).astype(
    feature_dtype)
indices = np.load(os.path.join(data_dir, indices_filename)).astype(
    np.int32)
indptr = np.load(os.path.join(data_dir, indptr_filename)).astype(
    np.int32)
labels = np.load(os.path.join(data_dir, labels_filename)).astype(
    np.int32)
shape = np.load(os.path.join(data_dir, shape_filename))

In [2]:
from scipy.sparse import csr_matrix

# Using sparse matrix to save space
col_ind = indices
row_ind = np.zeros((len(data)), dtype=np.int32)

for i in range(len(indptr) - 1):
    row_ind[indptr[i]: indptr[i + 1]] += i

features = csr_matrix((data, (row_ind, col_ind)), shape)

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.1, stratify=labels)
del features

In [4]:
import multiprocessing
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',
                    penalty='elasticnet',
                    max_iter=20,
                    n_jobs=(multiprocessing.cpu_count()-2))
clf.fit(X_train, y_train)
print('Test accuracy: %.3f' % clf.score(X_test, y_test))